# Object detection using Transfer Learning of CNN architectures

# Imports

In [1]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
from torchvision.datasets import CIFAR10
from torchvision.models import vgg16
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.nn import Module, Sequential, Linear, ReLU, CrossEntropyLoss

# Load Data

In [3]:
dataset = CIFAR10(root='dataset/', train=True, transform=ToTensor(), download=True)

In [4]:
train = []
test = []

for i in range(0,1000):
    train.append(dataset[i])
for i in range(1000, 1100):
    test.append(dataset[i])
    

In [5]:
train_loader = DataLoader(dataset=train, shuffle=True)
test_loader = DataLoader(dataset=test, shuffle=True)

# Load Pretained Model

In [6]:
model = vgg16(pretrained = True)

In [7]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

# Freeze Lower Layers

In [8]:
for p in model.parameters():
    p.requires_grad=False

In [9]:
class Identity(Module):
    def forward(self, x):
        return x

In [10]:
model.avgpool=Identity()

# Add custom classifier

In [11]:
model.classifier = Sequential(Linear(512,128), ReLU(), Linear(128,10))

# Train Custom Classifier

In [12]:
e = 5
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=1e-3)

In [13]:
for epoch in range(1,e+1):
    losses = []
    for data,target in train_loader:
        #forward
        scores = model(data)
        loss = criterion(scores,target)
        losses.append(loss.item())
        #backward
        optimizer.zero_grad()
        loss.backward()
        #adam's step
        optimizer.step()
    print("Cost at Epoch ",epoch, " is ", sum(losses)/len(losses))     

Cost at Epoch  1  is  1.7274823901942
Cost at Epoch  2  is  1.2056715849377215
Cost at Epoch  3  is  0.9032455919337372
Cost at Epoch  4  is  0.6813310442821694
Cost at Epoch  5  is  0.472728382048179


In [14]:
def check_accuracy(loader):
    correct = 0
    total = 0
    for data, target in loader:
        scores = model(data)
        _,pred = scores.max(1)
        if(pred == target):
            correct +=1
        total +=1
    print("Accuracy is : ", correct/total)

In [15]:
check_accuracy(train_loader)

Accuracy is :  0.914


In [16]:
check_accuracy(test_loader)

Accuracy is :  0.45


In [17]:
for data, target in test_loader:
    scores= model(data)
    _,pred = scores.max(1)
    print("Expected : ",target, "     Predicted : ", pred)

Expected :  tensor([8])      Predicted :  tensor([8])
Expected :  tensor([5])      Predicted :  tensor([3])
Expected :  tensor([4])      Predicted :  tensor([7])
Expected :  tensor([4])      Predicted :  tensor([4])
Expected :  tensor([8])      Predicted :  tensor([2])
Expected :  tensor([2])      Predicted :  tensor([4])
Expected :  tensor([6])      Predicted :  tensor([6])
Expected :  tensor([5])      Predicted :  tensor([5])
Expected :  tensor([3])      Predicted :  tensor([1])
Expected :  tensor([9])      Predicted :  tensor([9])
Expected :  tensor([9])      Predicted :  tensor([7])
Expected :  tensor([8])      Predicted :  tensor([4])
Expected :  tensor([1])      Predicted :  tensor([1])
Expected :  tensor([2])      Predicted :  tensor([5])
Expected :  tensor([3])      Predicted :  tensor([4])
Expected :  tensor([8])      Predicted :  tensor([8])
Expected :  tensor([3])      Predicted :  tensor([3])
Expected :  tensor([2])      Predicted :  tensor([0])
Expected :  tensor([9])     